In [1]:
import pandas as pd
from scipy import stats

In [3]:
# Đọc dữ liệu
df = pd.read_csv("subset-covid-data.csv")
df.head()

,country,continent,date,day,month,year,cases,deaths,country_code,population
0,Afghanistan,Asia,2020-04-12,12,4,2020,34,3,AFG,37172386.0
1,Albania,Europe,2020-04-12,12,4,2020,17,0,ALB,2866376.0
2,Algeria,Africa,2020-04-12,12,4,2020,64,19,DZA,42228429.0
3,Andorra,Europe,2020-04-12,12,4,2020,21,2,AND,77006.0
4,Angola,Africa,2020-04-12,12,4,2020,0,0,AGO,30809762.0


In [4]:
# Thông tin bộ dữ liệu
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country       206 non-null    object 
 1   continent     206 non-null    object 
 2   date          206 non-null    object 
 3   day           206 non-null    int64  
 4   month         206 non-null    int64  
 5   year          206 non-null    int64  
 6   cases         206 non-null    int64  
 7   deaths        206 non-null    int64  
 8   country_code  202 non-null    object 
 9   population    202 non-null    float64
dtypes: float64(1), int64(5), object(4)
memory usage: 16.2+ KB


In [5]:
# Mô tả bộ dữ liệu
df.describe()

,day,month,year,cases,deaths,population
count,206.000000,206.000000,206.0,206.000000,206.000000,2.020000e+02
mean,12.024272,3.985437,2020.0,395.247573,29.713592,3.731716e+07
std,0.424141,0.209020,0.0,2160.176439,159.533131,1.413776e+08
min,11.000000,1.000000,2020.0,0.000000,0.000000,1.000000e+03
25%,12.000000,4.000000,2020.0,0.000000,0.000000,1.278222e+06
50%,12.000000,4.000000,2020.0,11.500000,0.000000,7.042862e+06
75%,12.000000,4.000000,2020.0,88.750000,3.000000,2.600084e+07
max,18.000000,4.000000,2020.0,28391.000000,1831.000000,1.392730e+09


##### Có mối tương quan nào giữa số ca mắc và dân số ở các quốc gia hay không?
Do số ca nhiễm (cases) và dân số (population) đều là thuộc tính định lượng nên sử dụng kiểm định pearson.
- Giả thuyết không: không có mối tương quan tuyến tính giữa hai biến.
- Giả thuyết đối: không có mối tương quan tuyến tính giữa hai biến.

In [14]:
# Lọc dữ liệu
df_1 = df.filter(["cases", "population"])
df_1.head()

,cases,population
0,34,37172386.0
1,17,2866376.0
2,64,42228429.0
3,21,77006.0
4,0,30809762.0


In [11]:
# Xóa bỏ dữ liệu khuyết thiếu
df_1 = df_1.dropna()

In [12]:
r, pvalue = stats.pearsonr(df_1["cases"], df_1["population"])
print(f"r: {r}, pvalue: {pvalue}")

r: 0.17553926594294006, pvalue: 0.01246031070420893


- Nhận xét: do pvalue < 5% nên với mức ý nghĩa 5% bác bỏ giả thuyết không, chấp nhận giat thuyết đối.
- Kết luận: giữa dân số và số ca mắc có tương quan tuyến tính yếu với nhau.

##### Có mối tương quan nào giữa số ca mắc (cases) và số ca tử vong (deaths) hay không?
Do số ca nhiễm (cases) và số ca tử vong (deaths) đều là thuộc tính định lượng nên sử dụng kiểm định pearson.
- Giả thuyết không: không có mối tương quan tuyến tính giữa hai biến.
- Giả thuyết đối: không có mối tương quan tuyến tính giữa hai biến.

In [15]:
# Lọc dữ liệu
df_2 = df.filter(["cases", "deaths"])
df_2.head()

,cases,deaths
0,34,3
1,17,0
2,64,19
3,21,2
4,0,0


In [17]:
# Xóa bỏ dữ liệu khuyết thiếu
df_2 = df_2.dropna()

In [18]:
r, pvalue = stats.pearsonr(df_2["cases"], df_2["deaths"])
print(f"r: {r}, pvalue: {pvalue}")

r: 0.9432201374746727, pvalue: 1.3451948745846463e-99


- Nhận xét: do pvalue ~ 0, nên với mức ý nghĩa 5% bác bỏ giả thuyết không, chấp nhận giả thuyết đối.
- Kết luận: giữa số ca mắc và số ca tử vong có sự tương quan tuyến tính mạnh với nhau.

##### Biến đổi lại thuộc tính population về dạng thứ bậc thực hiện kiểm định tương quan giữa biến này với số ca nhiễm.

In [19]:
# Tiến hành tính các khoảng tứ phân vị của population
q1, q2, q3 = df_1["population"].quantile(0.25), df_1["population"].quantile(0.5), df_1["population"].quantile(0.75)

In [20]:
# Tiến hành biến đổi population
def population_order(population):
    if population < q1:
        return 1
    elif population >= q1 and population < q2:
        return 2
    elif population >= q2 and population < q3:
        return 3
    else:
        return 4
    
df_1["population_ordinal"] = df_1["population"].apply(population_order)
df_1.head()

,cases,population,population_ordinal
0,34,37172386.0,4
1,17,2866376.0,2
2,64,42228429.0,4
3,21,77006.0,1
4,0,30809762.0,4


Tiến hành kiểm định: Do population_ordinal là dữ liệu kiểu thứ bậc nên sử dụng kiểm định spearman để kiểm định tương quan giưa population_ordinal và cases.
- Giả thuyết không: giữa population_ordinal và cases không có tương quan.
- Giả thuyết đối: giữa population_ordinal và cases có tương quan.

In [21]:
r, pvalue = stats.spearmanr(df_1["cases"], df_1["population_ordinal"])
print(f"r: {r}, pvalue: {pvalue}")

r: 0.4437319255689447, pvalue: 2.3870927472609878e-11


pvalue ~ 0 và r ~ 0.5 chứng tỏ giữa thuộc tính population_ordinal và cases có tương quan với nhau. Như vậy có thể thấy, việc biến đổi dữ liệu thành các khoảng có thể làm nổi bật thông tin tương quan giữa các thuộc tính.